In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
from matplotlib import gridspec

import numpy as np
import scipy.sparse as sp
import numpy.linalg as la

In [4]:
# VECTOR<float,2> OPENGL_WORLD::
# Project_World_Space_Point_Onto_Image_Plane(const TV& world_space_point)
# {
#     MATRIX<float,4> perspective_matrix;
#     perspective_matrix(1,1)=(1/tan(.5f*fovy*(float)pi/180))*(float)window-      >Height()/(float)window->Width();
#     perspective_matrix(2,2)=1/tan(.5f*fovy*(float)pi/180);
#     perspective_matrix(3,3)=(farclip+nearclip)/(nearclip-farclip);
#     perspective_matrix(3,4)=(2*farclip*nearclip)/(nearclip-farclip);
#     perspective_matrix(4,3)=-1;

#     VECTOR<T,3> eye_space_point=TV(0.f,0.f,-camera_distance)+(arcball_matrix).Homogeneous_Times(world_space_point-target_position);
#     VECTOR<T,3> image_space_point=perspective_matrix.Homogeneous_Times(eye_space_point);
#     image_space_point(1)=((float)window->Width())*image_space_point(1)/2+((float)window->Width())/2;
#     image_space_point(2)=((float)window->Height())*image_space_point(2)/2+((float)window->Height())/2;
#     return VECTOR<float,2>(image_space_point(1),image_space_point(2));
# }

# https://www.opengl.org/sdk/docs/man2/xhtml/gluPerspective.xml
# https://unspecified.wordpress.com/2012/06/21/calculating-the-gluperspective-matrix-and-other-opengl-matrix-maths/
def perspective_matrix(fovY, aspect, near, far):
    print fovY
    print aspect
    f = 1.0 / np.tan (fovY / 2.0)
    p_matrix = np.matrix([[f/aspect, 0.0, 0.0, 0.0],
                          [0.0, f, 0.0, 0.0],
                          [0.0, 0.0, (far+near)/(near-far), (2.0*far*near)/(near-far)], 
                          [0.0, 0.0, -1.0, 0.0]])
    return p_matrix

# axes to camera space
def arcball_matrix(camForward, camUp):
    z = -camForward
    z = z / la.norm(z) # forward
    y = camUp # up
    x = np.cross(y, z) # right
    y = np.cross(z, x)
    x = x / la.norm(x)
    y = y / la.norm(y)
    return np.matrix([[x[0], x[1], x[2], 0.0], 
                      [y[0], y[1], y[2], 0.0],
                      [z[0], z[1], z[2], 0.0],
                      [0.0, 0.0, 0.0, 1.0]])
    # return np.matrix([[x[0], y[0], z[0], 0.0], 
    #                   [x[1], y[1], z[1], 0.0],
    #                   [x[2], y[2], z[2], 0.0],
    #                   [0.0, 0.0, 0.0, 1.0]])

def homogeneous_times(matrix, vec3):
    homogeneous_point = np.matrix([vec3[0], vec3[1], vec3[2], 1.0])
    vec4 = matrix * np.transpose(homogeneous_point)
    homogeneous_return = np.array([vec4[0]/vec4[3], vec4[1]/vec4[3], vec4[2]/vec4[3]])
    homogeneous_return = np.reshape(homogeneous_return, (1, 3))
    return np.array([homogeneous_return[0][0], homogeneous_return[0][1], homogeneous_return[0][2]])

In [12]:
# Conversion from world space to camera screen space
def world_to_screen(point, cam, forward, up, fovY, aspect):
    print "cam = ", cam
    print "forward = ", forward
    print "up = ", up

    proj = perspective_matrix(fovY, aspect, 0.1, 100.0)
    print "proj"
    print proj
    arcball = arcball_matrix(forward, up)
    print "arcball"
    print arcball

    look = cam + forward
    print PA-look
    eye_space_point = np.array([0.0, 0.0, -1.0]) + homogeneous_times(arcball, point - look)
    print "eye_space_point ", eye_space_point

    image_space_point = homogeneous_times(proj, eye_space_point)
    print "image_space_point ", image_space_point
    return np.array([image_space_point[0], image_space_point[1]])

# Apex shot (from Unity: z,x,-y)
# Apex - new camera position: (-39.6, 0.8, -92.7) orientation (0.0, 180.0, 0.0) forward (0.0, 0.0, -1.0) up (0.0, 1.0, 0.0)

# camera specs (position, forward, up, fov)
C = np.array([-92.9,-39.6,-0.8])
fwd = np.array([-1.0,0.0,0.0])
u = np.array([0.0,0.0,-1.0])
a = 16.0/9.0
fvY = 0.8726646

# scene specs
PA = np.array([-98.2,-40.6,-1.6])     # person A position
PB = np.array([-98.2,-38.6,-1.6])     # person B position

SA = world_to_screen(PA, C, fwd, u, fvY, a) # 0.220588 0.314124
SB = world_to_screen(PB, C, fwd, u, fvY, a) # -0.220588 0.314124

print "SA ", SA
print "SB ", SB

cam =  [-92.9 -39.6  -0.8]
forward =  [-1.  0.  0.]
up =  [ 0.  0. -1.]
0.8726646
1.77777777778
2.14450699329
proj
[[ 1.20628518  0.          0.          0.        ]
 [ 0.          2.14450699  0.          0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]
arcball
[[-0. -1. -0.  0.]
 [ 0.  0. -1.  0.]
 [ 1. -0. -0.  0.]
 [ 0.  0.  0.  1.]]
[-4.3 -1.  -0.8]
eye_space_point  [ 1.   0.8 -5.3]
image_space_point  [ 0.22760098  0.32369917  0.96422838]
cam =  [-92.9 -39.6  -0.8]
forward =  [-1.  0.  0.]
up =  [ 0.  0. -1.]
0.8726646
1.77777777778
2.14450699329
proj
[[ 1.20628518  0.          0.          0.        ]
 [ 0.          2.14450699  0.          0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]
arcball
[[-0. -1. -0.  0.]
 [ 0.  0. -1.  0.]
 [ 1. -0. -0.  0.]
 [ 0.  0.  0.  1.]]
[-4.3 -1.  -0.8]
eye_space_point  [-1.   0.8 -5.3]
image_space_point  [-0.22760098

In [13]:
# External shot (from Unity: z,x,-y)
# External - new camera position: (-42.6, 1.3, -96.5) orientation (0.0, 120.0, 0.0) forward (0.9, 0.0, -0.5) up (0.0, 1.0, 0.0)

# camera specs (position, forward, up, fov)
C_e = np.array([-96.5,-42.6,-1.3])
fwd_e = np.array([-0.5,0.9,0.0])
u_e = np.array([0.0,0.0,-1.0])

SA_e = world_to_screen(PA, C_e, fwd_e, u_e, fvY, a) # 0.2411121 0.2592802
SB_e = world_to_screen(PB, C_e, fwd_e, u_e, fvY, a) # -0.137811 0.1544899

print "SA ", SA_e
print "SB ", SB_e

cam =  [-96.5 -42.6  -1.3]
forward =  [-0.5  0.9  0. ]
up =  [ 0.  0. -1.]
0.8726646
1.77777777778
2.14450699329
proj
[[ 1.20628518  0.          0.          0.        ]
 [ 0.          2.14450699  0.          0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]
arcball
[[-0.87415728 -0.48564293 -0.          0.        ]
 [ 0.          0.         -1.          0.        ]
 [ 0.48564293 -0.87415728 -0.          0.        ]
 [ 0.          0.          0.          1.        ]]
[-1.2  1.1 -0.3]
eye_space_point  [ 0.51478151  0.3        -2.54434452]
image_space_point  [ 0.24406023  0.25285573  0.92331761]
cam =  [-96.5 -42.6  -1.3]
forward =  [-0.5  0.9  0. ]
up =  [ 0.  0. -1.]
0.8726646
1.77777777778
2.14450699329
proj
[[ 1.20628518  0.          0.          0.        ]
 [ 0.          2.14450699  0.          0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]
arcball
[[-0

In [14]:
# Topdown shot (from Unity: z,x,-y)
# TopDown - new camera position: (-39.6, 5.1, -95.7) orientation (60.0, 180.0, 0.0) forward (0.0, -0.9, -0.5) up (0.0, 0.5, -0.9)

# camera specs (position, forward, up, fov)
C_t = np.array([-95.7,-39.6,-5.1])
fwd_t = np.array([-0.5,0.0,0.9])
u_t = np.array([-0.9,0.0,-0.5])

SA_t = world_to_screen(PA, C_t, fwd_t, u_t, fvY, a) # 0.2838974 0.2176768
SB_t = world_to_screen(PB, C_t, fwd_t, u_t, fvY, a) # -0.2838974 0.2176768

print "SA ", SA_t 
print "SB ", SB_t 

cam =  [-95.7 -39.6  -5.1]
forward =  [-0.5  0.   0.9]
up =  [-0.9  0.  -0.5]
0.8726646
1.77777777778
2.14450699329
proj
[[ 1.20628518  0.          0.          0.        ]
 [ 0.          2.14450699  0.          0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]
arcball
[[-0.         -1.          0.          0.        ]
 [-0.87415728  0.         -0.48564293  0.        ]
 [ 0.48564293 -0.         -0.87415728  0.        ]
 [ 0.          0.          0.          1.        ]]
[-2.  -1.   2.6]
eye_space_point  [ 1.          0.48564293 -4.24409478]
image_space_point  [ 0.28422673  0.24539147  0.95483053]
cam =  [-95.7 -39.6  -5.1]
forward =  [-0.5  0.   0.9]
up =  [-0.9  0.  -0.5]
0.8726646
1.77777777778
2.14450699329
proj
[[ 1.20628518  0.          0.          0.        ]
 [ 0.          2.14450699  0.          0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]
arcbal